# Installing and starting mysql

In [ ]:
! apt-get update
! apt-get install mysql-server
!sudo service mysql stop
!sudo usermod -d /var/lib/mysql/ mysql
!sudo service mysql start
!mysql --version   # Making sure everything has been installed correctly by checking the version
!service mysql start   # Starting our server

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,673 kB]
Get:13 https://developer.download.nvidia.com/c

# Import Books from Kaggle

In [ ]:
import kagglehub
path = kagglehub.dataset_download("trnnhtminh/goodread-books-ratings")
print("Path to dataset files:", path)
!ls $path

100%|██████████| 1.48G/1.48G [00:17<00:00, 89.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/trnnhtminh/goodread-books-ratings/versions/1
merged_books_ratings.csv


In [ ]:
!mv {path}/merged_books_ratings.csv /var/lib/mysql-files/
!ls /var/lib/mysql-files/
!head -n10 /var/lib/mysql-files/merged_books_ratings.csv

merged_books_ratings.csv
user_id,isbn,rating,authors,description,genres,pages,title,publishYear,publishMonth,publishDay
1,0143034901,5,carlos-ruiz-zafn lucia-graves,barcelona city slowly heals war wound daniel antiquarian book dealer son mourns loss mother find solace mysterious book entitled shadow wind one julian carax set find author work make shocking discovery someone systematically destroying every copy every book carax written fact daniel may last caraxs book existence soon daniel seemingly innocent quest open door one barcelona darkest secret -- epic story murder madness doomed love -- back cover,fiction historical-fiction mystery fantasy,487.0,shadow wind cemetery forgotten book,2005,10,28
1,0451529308,3,louisa-may-alcott,generation reader young old male female fallen love march sister louisa may alcott popular enduring novel little woman talented tomboy author-to-be jo tragically frail beth beautiful meg romantic spoiled amy united devotion struggle survive new england civil 

# Create and populate the Books table in mysql

In [ ]:
!mysql -e "create database Books;"
!mysql -D "Books" -e "create table books (ID INT AUTO_INCREMENT PRIMARY KEY, user_id VARCHAR(255), isbn VARCHAR(255), rating INT, authors TEXT, description TEXT, genres TEXT, pages INT, title TEXT, publishYear INT, publishMonth INT, publishDay INT);"
!mysql -D "Books" -e "describe books;"

ERROR 1007 (HY000) at line 1: Can't create database 'Books'; database exists
ERROR 1050 (42S01) at line 1: Table 'books' already exists
+--------------+--------------+------+-----+---------+----------------+
| Field        | Type         | Null | Key | Default | Extra          |
+--------------+--------------+------+-----+---------+----------------+
| ID           | int          | NO   | PRI | NULL    | auto_increment |
| user_id      | varchar(255) | YES  |     | NULL    |                |
| isbn         | varchar(255) | YES  |     | NULL    |                |
| rating       | int          | YES  |     | NULL    |                |
| authors      | text         | YES  |     | NULL    |                |
| description  | text         | YES  |     | NULL    |                |
| genres       | text         | YES  |     | NULL    |                |
| pages        | int          | YES  |     | NULL    |                |
| title        | text         | YES  |     | NULL    |                |


In [ ]:
!time mysql -D "Books" -e "LOAD DATA INFILE '/var/lib/mysql-files/merged_books_ratings.csv' IGNORE INTO TABLE books FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '\"' LINES TERMINATED BY '\n' IGNORE 1 ROWS (user_id, isbn, rating, authors, description, genres, pages, title, publishYear, publishMonth, publishDay);";
!mysql -D "Books" -e "select * from books limit 10"
!mysql -D "Books" -e "select count(*) from books"
!mysql -D "Books" -e "select count(distinct isbn) from books"


real	8m30.819s
user	0m0.005s
sys	0m0.005s
+----+---------+------------+--------+-------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Create a junction table (isbn <-> genre) named Genres

In [ ]:
!time mysql -D "Books" -e "select distinct isbn, genres into outfile '/var/lib/mysql-files/isbn_genres.csv' FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' from books"
!head -n10 /var/lib/mysql-files/isbn_genres.csv


real	0m21.548s
user	0m0.004s
sys	0m0.005s
0143034901,fiction historical-fiction mystery fantasy
0451529308,classic fiction historical-fiction young-adult romance
0812550706,science-fiction fiction young-adult fantasy classic
1580493882,classic fiction
0142001740,fiction historical-fiction young-adult classic contemporary chick-lit
0812550757,science-fiction fiction fantasy young-adult
0061120081,classic fiction historical-fiction young-adult
0375751513,classic fiction horror fantasy
0099408392,fiction classic fantasy young-adult
0142000663,classic fiction historical-fiction


In [ ]:
import pandas as pd
import time

start = time.perf_counter()

input_file = '/var/lib/mysql-files/isbn_genres.csv'
output_file = '/var/lib/mysql-files/m2m_isbn_genres.csv'

input_df = pd.read_csv(input_file)
output_df = pd.DataFrame(columns=['isbn', 'genre'])

for index, series in input_df.iterrows():
  for genre in series[1].split():
    output_df.loc[len(output_df)] = [series[0].strip(), genre.strip()]
output_df.to_csv(output_file, index=False)

end = time.perf_counter()
print("Execution time: " + str(end - start))

Streaming output truncated to the last 5000 lines.
<ipython-input-10-1753b836adcd>:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output_df.loc[len(output_df)] = [series[0].strip(), genre.strip()]
<ipython-input-10-1753b836adcd>:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output_df.loc[len(output_df)] = [series[0].strip(), genre.strip()]
<ipython-input-10-1753b836adcd>:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  outpu

Execution time: 77.94801557899996


<ipython-input-10-1753b836adcd>:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output_df.loc[len(output_df)] = [series[0].strip(), genre.strip()]
<ipython-input-10-1753b836adcd>:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for genre in series[1].split():
<ipython-input-10-1753b836adcd>:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output_df.loc[len(output_df)] = [series[0].strip(), genre.strip()]
<ipython-input-10-1753b8

Execution time is hidden among the last 20 lines of output. It reads:
`Execution time: 77.94801557899996`

In [ ]:
!head -n10 '/var/lib/mysql-files/m2m_isbn_genres.csv'
!wc -l '/var/lib/mysql-files/m2m_isbn_genres.csv'

isbn,genre
0451529308,classic
0451529308,fiction
0451529308,historical-fiction
0451529308,young-adult
0451529308,romance
0812550706,science-fiction
0812550706,fiction
0812550706,young-adult
0812550706,fantasy
42477 /var/lib/mysql-files/m2m_isbn_genres.csv


In [ ]:
!mysql -D "Books" -e "drop table genres"
!mysql -D "Books" -e "create table genres (isbn varchar(255), genre varchar(255), PRIMARY KEY(isbn, genre));"
!mysql -D "Books" -e "describe genres;"

ERROR 1051 (42S02) at line 1: Unknown table 'Books.genres'
+-------+--------------+------+-----+---------+-------+
| Field | Type         | Null | Key | Default | Extra |
+-------+--------------+------+-----+---------+-------+
| isbn  | varchar(255) | NO   | PRI | NULL    |       |
| genre | varchar(255) | NO   | PRI | NULL    |       |
+-------+--------------+------+-----+---------+-------+


In [ ]:
!time mysql -D "Books" -e "LOAD DATA INFILE '/var/lib/mysql-files/m2m_isbn_genres.csv' INTO TABLE genres FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' ignore 1 rows (isbn, genre);";
!mysql -D "Books" -e "select * from genres limit 10";
!mysql -D "Books" -e "select count(*) from genres"


real	0m1.002s
user	0m0.005s
sys	0m0.003s
+------------+--------------+
| isbn       | genre        |
+------------+--------------+
| 000100039X | classic      |
| 000100039X | fiction      |
| 000100039X | philosophy   |
| 000100039X | poetry       |
| 000100039X | religion     |
| 000100039X | self-help    |
| 000100039X | spirituality |
| 0001713221 | classic      |
| 0001713221 | fiction      |
| 0001713221 | young-adult  |
+------------+--------------+
+----------+
| count(*) |
+----------+
|    42476 |
+----------+


# Produce the genre <-> rating table named Ratings

In [ ]:
!mysql -D "Books" -e "drop table ratings"
!mysql -D "Books" -e "create table ratings (ID INT AUTO_INCREMENT PRIMARY KEY, genre varchar(255), rating int);"
!mysql -D "Books" -e "describe ratings"

ERROR 1051 (42S02) at line 1: Unknown table 'Books.ratings'
+--------+--------------+------+-----+---------+----------------+
| Field  | Type         | Null | Key | Default | Extra          |
+--------+--------------+------+-----+---------+----------------+
| ID     | int          | NO   | PRI | NULL    | auto_increment |
| genre  | varchar(255) | YES  |     | NULL    |                |
| rating | int          | YES  |     | NULL    |                |
+--------+--------------+------+-----+---------+----------------+


In [ ]:
!time mysql -D "Books" -e "insert into ratings (genre, rating) SELECT G.genre, B.rating FROM genres G JOIN books B ON G.isbn = B.isbn;"
!mysql -D "Books" -e "select * from ratings limit 10"
!mysql -D "Books" -e "select count(distinct genre), count(distinct rating) from ratings"


real	5m35.254s
user	0m0.008s
sys	0m0.003s
+----+--------------------+--------+
| ID | genre              | rating |
+----+--------------------+--------+
|  1 | classic            |      3 |
|  2 | fiction            |      3 |
|  3 | historical-fiction |      3 |
|  4 | romance            |      3 |
|  5 | young-adult        |      3 |
|  6 | classic            |      5 |
|  7 | fantasy            |      5 |
|  8 | fiction            |      5 |
|  9 | science-fiction    |      5 |
| 10 | young-adult        |      5 |
+----+--------------------+--------+
+-----------------------+------------------------+
| count(distinct genre) | count(distinct rating) |
+-----------------------+------------------------+
|                    39 |                      5 |
+-----------------------+------------------------+


# Find the most popular genre!

In [ ]:
!mysql -D "Books" -e "drop table ranking;"

In [ ]:
!mysql -D "Books" -e "create table ranking (ID INT AUTO_INCREMENT PRIMARY KEY, popularity int, genre varchar(255));"
!mysql -D "Books" -e "describe ranking"

+------------+--------------+------+-----+---------+----------------+
| Field      | Type         | Null | Key | Default | Extra          |
+------------+--------------+------+-----+---------+----------------+
| ID         | int          | NO   | PRI | NULL    | auto_increment |
| popularity | int          | YES  |     | NULL    |                |
| genre      | varchar(255) | YES  |     | NULL    |                |
+------------+--------------+------+-----+---------+----------------+


We intended to compute popularity as count(\*) \* avg(rating). However, it occurred to us later that this is just equal to sum(rating), as avg(rating) == sum(rating)/count(\*)

In [ ]:
!time mysql -D "Books" -e "insert into ranking (popularity, genre) select sum(rating), genre from ratings group by genre"
!time mysql -D "Books" -e "select popularity, genre from ranking order by popularity desc;"


real	0m56.919s
user	0m0.003s
sys	0m0.006s
+------------+--------------------+
| popularity | genre              |
+------------+--------------------+
|   19869613 | fiction            |
|    9885284 | fantasy            |
|    8557100 | classic            |
|    7507825 | young-adult        |
|    6331677 | romance            |
|    5370340 | contemporary       |
|    5210228 | historical-fiction |
|    4622843 | mystery            |
|    3638978 | science-fiction    |
|    3376067 | thriller           |
|    2705488 | crime              |
|    2588188 | paranormal         |
|    2548798 | suspense           |
|    2329726 | nonfiction         |
|    2117020 | chick-lit          |
|    1996196 | horror             |
|    1415946 | biography          |
|    1340160 | history            |
|    1203974 | memoir             |
|    1176248 | philosophy         |
|    1070730 | book               |
|     810084 | poetry             |
|     801236 | psychology         |
|     750382 | religi

# For checkpoint purposes

In [ ]:
# !mysql -D "Books" -e "select * into outfile '/var/lib/mysql-files/Books.csv' FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' from books"
# !mysql -D "Books" -e "select * into outfile '/var/lib/mysql-files/Genres.csv' FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' from genres"
# !mysql -D "Books" -e "select * into outfile '/var/lib/mysql-files/Ratings.csv' FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' from ratings"
# !mysql -D "Books" -e "select * into outfile '/var/lib/mysql-files/Ranking.csv' FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' from ranking order by popularity desc"

In [ ]:
# from google.colab import files

# files.download('/var/lib/mysql-files/Books.csv')
# files.download('/var/lib/mysql-files/Genres.csv')
# files.download('/var/lib/mysql-files/Ratings.csv')
# files.download('/var/lib/mysql-files/Ranking.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>